In [1]:
!pip install catboost

In [58]:
import pandas as pd
from catboost import CatBoostClassifier, Pool, cv
from tqdm.notebook import tqdm

In [3]:
train_data = pd.read_csv('fewshot_embed_train.csv')

In [4]:
test_data = pd.read_csv('fewshot_embed_test.csv')

In [5]:
train_data

,Исполнитель,Группа тем,Текст инцидента,Тема,0,1,2,3,4,5,...,nli_Строительство и архитектура,nli_Экономика и бизнес,nli_Физическая культура и спорт,nli_Связь и телевидение,nli_Газ и топливо,nli_Государственная собственность,nli_Торговля,nli_Памятники и объекты культурного наследия,nli_Погребение и похоронное дело,nli_Мобилизация
0,Город Пермь,Погребение и похоронное дело,Погребения - это серьезная проблема в нашей ст...,Погребение и похоронное дело,-0.294947,-0.055111,-0.917189,0.076128,-0.168686,0.458902,...,0.021046,0.004908,0.044621,0.049851,0.026472,0.016191,0.014407,0.049282,0.131090,0.009004
1,Город Пермь,Погребение и похоронное дело,"Я также считаю, что мы должны делать все возмо...",Погребение и похоронное дело,-0.215702,0.045419,-1.404945,-0.113390,-0.199697,0.628931,...,0.012543,0.001811,0.016096,0.046435,0.021829,0.015271,0.006325,0.051986,0.159060,0.002771
2,Город Пермь,Погребение и похоронное дело,"Я думаю, что многие люди уже готовы переходить...",Погребение и похоронное дело,-0.074277,-0.069510,-1.307976,-0.731061,-0.117746,0.112123,...,0.009973,0.005146,0.048277,0.050400,0.052558,0.007187,0.013559,0.019356,0.181261,0.004151
3,Город Пермь,Погребение и похоронное дело,"Я считаю, что строительство новых крематориев ...",Погребение и похоронное дело,-0.215675,-0.010250,-1.629285,-0.177629,-0.034335,0.769009,...,0.014146,0.031556,0.038072,0.049106,0.050131,0.011899,0.060701,0.071414,0.093479,0.005136
4,Город Пермь,Погребение и похоронное дело,"Я думаю, что мы должны обсуждать этот вопрос о...",Погребение и похоронное дело,0.091469,-0.337995,-1.098347,-0.416570,0.188040,0.489746,...,0.026353,0.027623,0.140774,0.039312,0.060938,0.011117,0.049114,0.017505,0.031063,0.011791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16847,ИГЖН ПК,ЖКХ,"Счёт за отопление пришёл за 1,5 месяца октябр...",Завышение платы за коммунальные услуги,0.494950,-0.152662,-1.038108,-0.324845,0.389979,0.587082,...,0.021880,0.028506,0.040916,0.023414,0.170164,0.013231,0.073731,0.011126,0.003037,0.004261
16848,Министерство социального развития ПК,Социальное обслуживание и защита,"Здравствуйте, справку о том что я не получала...",Оказание гос. соц. помощи,-0.245126,-0.062770,-0.160016,0.977306,-0.091960,1.371442,...,0.004896,0.018229,0.063768,0.062380,0.023841,0.017222,0.029269,0.019695,0.004285,0.003090
16849,ИГЖН ПК,ЖКХ,"Добрый день, куда и к кому обращаться о ненад...",Ненадлежащее содержание и эксплуатация МКД,0.096977,0.334582,0.256389,0.675268,-0.216912,0.327004,...,0.038565,0.002348,0.047807,0.013241,0.008335,0.007816,0.010814,0.016424,0.010363,0.001689
16850,ИГЖН ПК,ЖКХ,"Приморский 19, управляйка не чистит снег с кр...",★ Наледь и сосульки на кровле,-0.198398,-0.084078,-0.520404,-1.223011,0.486935,0.555129,...,0.034284,0.013381,0.068992,0.009127,0.011861,0.007010,0.023945,0.028093,0.005709,0.005388


In [6]:
groups = list(train_data['Группа тем'].unique())
themes = []
for g in groups:
    themes.append(list(train_data[train_data['Группа тем'] == g]['Тема'].unique()))
gr_th_dict = dict(zip(groups, themes))
gr_th_dict

{'Погребение и похоронное дело': ['Погребение и похоронное дело'],
 'МФЦ "Мои документы"': ['Государственные услуги', 'МФЦ "Мои документы"'],
 'Торговля': ['★ Нестационарная торговля (киоски, павильоны, сезонная торговля)'],
 'Государственная собственность': ['Региональное имущество'],
 'Памятники и объекты культурного наследия': ['Памятники и объекты культурного наследия'],
 'Роспотребнадзор': ['Санитарно-эпидемиологическое благополучие',
  'Обработка и уничтожение грызунов (дератизация)'],
 'Экология': ['Выбросы вредных веществ в атмосферу/загрязнение воздуха',
  'Выбросы вредных веществ в водоёмы/загрязнение водоёмов'],
 'Экономика и бизнес': ['Трудоустройство', 'Цены и ценообразование'],
 'Электроснабжение': ['Качество электроснабжения', '★ Повреждение опор ЛЭП'],
 'Культура': ['Учреждения культуры', 'Культурно-досуговые мероприятия'],
 'Спецпроекты': ['Спецпроекты'],
 'Газ и топливо': ['Восстановление газоснабжения',
  'Запрос на газификацию и её условия',
  'Сроки газификации',
 

In [7]:
cols_drop = ['Исполнитель', 'Группа тем', 'Текст инцидента', 'Тема']

In [8]:
train_data_X = train_data.drop(cols_drop, axis=1)
train_data_Y_group = train_data['Группа тем']
train_data_Y_theme = train_data['Тема']
train_data_Y_performer = train_data['Исполнитель']

In [9]:
test_data_X = test_data.drop(cols_drop, axis=1)
test_data_Y_group = test_data['Группа тем']
test_data_Y_theme = test_data['Тема']
test_data_Y_performer = test_data['Исполнитель']

In [10]:
pool_train_group = Pool(train_data_X, train_data_Y_group)
pool_train_theme = Pool(train_data_X, train_data_Y_theme)
pool_train_performer = Pool(train_data_X, train_data_Y_performer)

In [11]:
pool_test_group = Pool(test_data_X, test_data_Y_group)
pool_test_theme = Pool(test_data_X, test_data_Y_theme)
pool_test_performer = Pool(test_data_X, test_data_Y_performer)

In [12]:
cbt_cl_group = CatBoostClassifier(loss_function='MultiClass', eval_metric='TotalF1', task_type="GPU", devices='0:1')

In [13]:
cbt_cl_theme = CatBoostClassifier(loss_function='MultiClass', eval_metric='TotalF1', task_type="GPU", devices='0:1')

In [14]:
cbt_cl_performer = CatBoostClassifier(loss_function='MultiClass', eval_metric='TotalF1', task_type="GPU", devices='0:1')

In [15]:
cbt_cl_group.fit(pool_train_group, verbose=10, plot=False)

Learning rate set to 0.115484
0:	learn: 0.3895105	total: 70.9ms	remaining: 1m 10s
10:	learn: 0.5251332	total: 707ms	remaining: 1m 3s
20:	learn: 0.6051698	total: 1.34s	remaining: 1m 2s
30:	learn: 0.6530631	total: 1.95s	remaining: 1m
40:	learn: 0.6869556	total: 2.57s	remaining: 1m
50:	learn: 0.7135636	total: 3.18s	remaining: 59.2s
60:	learn: 0.7329382	total: 3.78s	remaining: 58.2s
70:	learn: 0.7482839	total: 4.39s	remaining: 57.5s
80:	learn: 0.7607088	total: 4.98s	remaining: 56.5s
90:	learn: 0.7717177	total: 5.58s	remaining: 55.8s
100:	learn: 0.7807825	total: 6.16s	remaining: 54.9s
110:	learn: 0.7868425	total: 6.74s	remaining: 54s
120:	learn: 0.7951547	total: 7.33s	remaining: 53.3s
130:	learn: 0.8006607	total: 7.92s	remaining: 52.6s
140:	learn: 0.8066810	total: 8.49s	remaining: 51.7s
150:	learn: 0.8111125	total: 9.06s	remaining: 50.9s
160:	learn: 0.8153454	total: 9.61s	remaining: 50.1s
170:	learn: 0.8208332	total: 10.2s	remaining: 49.3s
180:	learn: 0.8254865	total: 10.7s	remaining: 48.6s

In [18]:
cbt_cl_group.save_model('./cb_models/model_group')

In [19]:
cbt_cl_theme.fit(pool_train_theme, verbose=10, plot=False)

Learning rate set to 0.115484
0:	learn: 0.1143796	total: 548ms	remaining: 9m 7s
10:	learn: 0.2148538	total: 5.86s	remaining: 8m 47s
20:	learn: 0.2885345	total: 11.1s	remaining: 8m 38s
30:	learn: 0.3426094	total: 16.3s	remaining: 8m 29s
40:	learn: 0.3926094	total: 21.6s	remaining: 8m 24s
50:	learn: 0.4374609	total: 26.8s	remaining: 8m 19s
60:	learn: 0.4759174	total: 32s	remaining: 8m 12s
70:	learn: 0.5141458	total: 37.1s	remaining: 8m 5s
80:	learn: 0.5530390	total: 42.3s	remaining: 7m 59s
90:	learn: 0.5800909	total: 47.5s	remaining: 7m 54s
100:	learn: 0.6052809	total: 52.5s	remaining: 7m 47s
110:	learn: 0.6320335	total: 57.5s	remaining: 7m 40s
120:	learn: 0.6553767	total: 1m 2s	remaining: 7m 34s
130:	learn: 0.6743533	total: 1m 7s	remaining: 7m 26s
140:	learn: 0.6946449	total: 1m 12s	remaining: 7m 21s
150:	learn: 0.7134997	total: 1m 17s	remaining: 7m 14s
160:	learn: 0.7317925	total: 1m 22s	remaining: 7m 9s
170:	learn: 0.7486753	total: 1m 27s	remaining: 7m 2s
180:	learn: 0.7619953	total: 

In [20]:
cbt_cl_theme.save_model('./cb_models/model_theme')

In [21]:
cbt_cl_performer.fit(pool_train_performer, verbose=10, plot=False)

Learning rate set to 0.115484
0:	learn: 0.4437360	total: 35.3ms	remaining: 35.2s
10:	learn: 0.5180268	total: 300ms	remaining: 27s
20:	learn: 0.5657303	total: 563ms	remaining: 26.2s
30:	learn: 0.5933810	total: 831ms	remaining: 26s
40:	learn: 0.6128019	total: 1.09s	remaining: 25.5s
50:	learn: 0.6270623	total: 1.35s	remaining: 25.1s
60:	learn: 0.6412215	total: 1.61s	remaining: 24.9s
70:	learn: 0.6542841	total: 1.87s	remaining: 24.5s
80:	learn: 0.6675625	total: 2.13s	remaining: 24.2s
90:	learn: 0.6787879	total: 2.39s	remaining: 23.9s
100:	learn: 0.6888669	total: 2.64s	remaining: 23.5s
110:	learn: 0.6990691	total: 2.9s	remaining: 23.3s
120:	learn: 0.7084376	total: 3.15s	remaining: 22.9s
130:	learn: 0.7179942	total: 3.4s	remaining: 22.6s
140:	learn: 0.7261500	total: 3.65s	remaining: 22.2s
150:	learn: 0.7341246	total: 3.9s	remaining: 21.9s
160:	learn: 0.7426396	total: 4.15s	remaining: 21.6s
170:	learn: 0.7513813	total: 4.4s	remaining: 21.3s
180:	learn: 0.7588060	total: 4.64s	remaining: 21s
19

In [22]:
cbt_cl_performer.save_model('./cb_models/model_performer')

In [23]:
theme_classes = cbt_cl_theme.classes_

In [62]:
def get_predict(x, proba_flag=False, perf_flag=False):
    #pool_x = Pool(x)
    # GROUP
    pred_group = cbt_cl_group.predict(x)[0]
    # print(pred_group)
    # THEME
    if proba_flag:
        theme_proba = cbt_cl_theme.predict_proba(x)
        theme_dict = dict(zip(theme_classes, theme_proba))
        theme_keys = gr_th_dict[pred_group]
        res = {}
        for el in theme_keys:
            res[el] = theme_dict[el]
        max_val = max(res.values())
        res = {k:v for k, v in res.items() if v == max_val}
        pred_theme = list(res.keys())[0]
    else:
        pred_theme = cbt_cl_theme.predict(x)[0]
    #print(pred_theme)
    if perf_flag:
        pred_performer = cbt_cl_performer.predict(x)[0]
        #print(pred_performer)
        result = {'Группа тем': pred_group, 'Тема': pred_theme, 'Исполнитель': pred_performer}
    else:
        result = {'Группа тем': pred_group, 'Тема': pred_theme,  'Исполнитель': 'НЕТ'}
    return result

In [56]:
x = test_data_X.loc[0].values
x

array([ 0.0314665 , -0.08282307,  1.23735678, ...,  0.01618499,
        0.02247923,  0.00803234])

In [57]:
print(get_predict(x, True, True))

{'Группа тем': 'ЖКХ', 'Тема': 'Жалобы на управляющие компании', 'Исполнитель': 'ИГЖН ПК'}


In [69]:
df_test = test_data_X.copy()
print(len(df_test))
df_test[['Группа тем', 'Тема', 'Исполнитель']] = ''
for index, row in tqdm(test_data_X.iterrows()):
    predictions = get_predict(row.values)
    #print(predictions)
    df_test.loc[index, 'Группа тем'] = predictions['Группа тем']
    df_test.loc[index, 'Тема'] = predictions['Тема']
    df_test.loc[index, 'Исполнитель'] = predictions['Исполнитель']
    #print(df_test)

7435


0it [00:00, ?it/s]

In [70]:
df_test

,0,1,2,3,4,5,6,7,8,9,...,nli_Связь и телевидение,nli_Газ и топливо,nli_Государственная собственность,nli_Торговля,nli_Памятники и объекты культурного наследия,nli_Погребение и похоронное дело,nli_Мобилизация,Группа тем,Тема,Исполнитель
0,0.031466,-0.082823,1.237357,1.026363,0.122704,-0.073226,-0.020305,-0.708686,0.049879,0.180117,...,0.129245,0.059134,0.022013,0.073064,0.016185,0.022479,0.008032,ЖКХ,Жалобы на управляющие компании,НЕТ
1,0.548626,0.551490,0.975990,0.684661,0.433979,-1.009545,0.474450,0.386742,0.220841,0.470633,...,0.032441,0.094760,0.013543,0.060041,0.019953,0.003712,0.003704,Физическая культура и спорт,Оказание гос. соц. помощи,НЕТ
2,0.239955,0.009783,-1.173516,-0.723214,0.257519,0.285644,0.683143,0.200508,-0.496863,0.512365,...,0.050202,0.035142,0.020499,0.015097,0.039281,0.000747,0.005815,Дороги,Ремонт дороги,НЕТ
3,0.463223,-0.115075,-0.149745,0.439815,0.407077,-0.751671,-0.610838,-0.737995,-0.058724,1.081843,...,0.024520,0.020061,0.013857,0.027733,0.054564,0.009781,0.014844,Здравоохранение/Медицина,Нехватка или сокращение врачей и медицинских у...,НЕТ
4,-0.442561,-0.162223,-0.025691,-0.388856,0.081860,0.562882,-0.203205,-0.855122,-0.824697,1.152326,...,0.032979,0.036513,0.029055,0.009825,0.051661,0.008265,0.020726,Дороги,★ Нарушение правил очистки дорог от снега и на...,НЕТ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7430,-0.542317,0.065096,-0.010285,-0.148252,-0.020627,0.769689,0.072186,-0.391850,0.157679,0.860796,...,0.094834,0.005576,0.010169,0.014481,0.017956,0.007221,0.017121,Социальное обслуживание и защита,Оказание гос. соц. помощи,НЕТ
7431,0.749674,0.069271,-0.290126,-0.823492,0.469242,0.903284,0.189811,-0.385004,-0.007156,0.148818,...,0.021785,0.010483,0.011265,0.057001,0.023589,0.016975,0.007541,ЖКХ,★ Уборка/Вывоз мусора,НЕТ
7432,-0.365803,-0.111526,-1.076791,-1.691728,0.023753,0.996946,-1.036180,0.086070,-0.500380,0.177465,...,0.040737,0.012675,0.010667,0.007600,0.017869,0.009347,0.008683,Здравоохранение/Медицина,★ Оказание медицинской помощи не в полном объе...,НЕТ
7433,0.102735,0.196335,-1.074159,-0.443382,0.560598,-0.422613,0.276889,0.195564,-0.416977,-0.534513,...,0.039190,0.019462,0.030107,0.014063,0.033821,0.002108,0.003395,Здравоохранение/Медицина,Нехватка или сокращение врачей и медицинских у...,НЕТ
